<a href="https://colab.research.google.com/github/Poojith2602/An-NLP-Powered-Chatbot-with-Integrated-Text-Summarization-/blob/main/NLP_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import nltk
nltk.download('punkt_tab')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
import nltk
import os
from google.colab import files

uploaded = files.upload()
print("len(uploaded.keys()):", len(uploaded.keys()))

# Load text into an object called "text"
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    with open(fn, 'r', encoding='utf8', errors='ignore') as f:
        # set contents of file to your text object HERE
        text = f.read()
        f.seek(0)  # Move the file pointer to the beginning of the file
        partial_text = f.read(100)  # Read the first 100 characters
print(partial_text)


Saving chatbot.txt to chatbot.txt
len(uploaded.keys()): 1
User uploaded file "chatbot.txt" with length 966 bytes

A chatbot or chatterbot is a software application aimed at simulating conversation with human users


In [4]:
sent_tokens = nltk.sent_tokenize(text)# converts to list of sentences
word_tokens = nltk.word_tokenize(text)# converts to list of words
def tokenize_sentences(text):
    sentences = text

    return nltk.sent_tokenize(text)

sentences = tokenize_sentences(text)

In [5]:
import string

translation = str.maketrans('', '', string.punctuation)

def preprocess_text(sentences):

    cleaned = [w.lower() for w in sentences]
    cleaned = [w.translate(translation) for w in cleaned]

    return cleaned

cleaned = preprocess_text(sentences)

In [6]:
# create TFIDF (or BOW) matrix on SENTENCES
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVec = TfidfVectorizer(stop_words='english') #see documentation for options!
tfidf = TfidfVec.fit_transform(cleaned)

X = pd.DataFrame(tfidf.toarray(), columns = TfidfVec.get_feature_names_out(), dtype='float32')
print(X.head())

       1994  accessed  acquisition     aimed  application  apps  assistant  \
0  0.000000       0.0     0.000000  0.310414     0.310414   0.0        0.0   
1  0.000000       0.0     0.000000  0.000000     0.000000   0.0        0.0   
2  0.000000       0.0     0.287854  0.000000     0.000000   0.0        0.0   
3  0.000000       0.0     0.000000  0.000000     0.000000   0.0        0.0   
4  0.310414       0.0     0.000000  0.000000     0.000000   0.0        0.0   

   assistants    behave   chatbot  ...    turing  typically       use  \
0         0.0  0.000000  0.310414  ...  0.000000   0.000000  0.000000   
1         0.0  0.320132  0.000000  ...  0.320132   0.000000  0.000000   
2         0.0  0.000000  0.000000  ...  0.000000   0.287854  0.000000   
3         0.0  0.000000  0.000000  ...  0.000000   0.000000  0.217863   
4         0.0  0.000000  0.000000  ...  0.000000   0.000000  0.000000   

       used     users   various    verbot  virtual  websites   wording  
0  0.000000  0.3104

In [7]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):

    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [8]:
def respond(user_input):
    bot_response = ''
    query = TfidfVec.transform([user_input])
    cosine_sim = query.dot(X.T)
    max_sim_index = cosine_sim.argmax()
    if max_sim_index == 0:
        return 'BOT: I beg your pardon? I\'m not quite sure I got your meaning.'
    bot_response = sentences[max_sim_index]
    return bot_response

In [9]:
from heapq import nlargest

In [10]:
def summarize_text(user_input):
    if user_input.lower() == "~summarize":
        import spacy
        from spacy.lang.en.stop_words import STOP_WORDS
        from string import punctuation
        from heapq import nlargest
        stopwords = list(STOP_WORDS)
        nlp = spacy.load('en_core_web_sm')
        doc = nlp(text)

        word_frequencies = {}
        for word in doc:
            if word.text.lower() not in stopwords and word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1

        max_frequency = max(word_frequencies.values())
        for word in word_frequencies.keys():
            word_frequencies[word] = word_frequencies[word] / max_frequency

        sentence_tokens = [sent for sent in doc.sents]
        sentence_scores = {}
        for sent in sentence_tokens:
            for word in sent:
                if word.text.lower() in word_frequencies.keys():
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

        select_length = int(len(sentence_tokens) * 0.3)
        summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)

        final_summary = [word.text for word in summary]
        summary_text = ' '.join(final_summary)

        print(f"\nInitial number of characters: {len(text)}")
        print(f"Number of characters in the summarized text: {len(summary_text)}\n")

        return summary_text

    return None

In [ ]:
# Main chatbot logic
flag = True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")

while flag:
    user_response = input("YOU: ")
    user_response = user_response.lower()

    if user_response != 'bye':
        if user_response == 'thanks' or user_response == 'thank you':
            flag = False
            print("ROBO: You are welcome.")
        else:
            # Check for summarization command
            summary_result = summarize_text(user_response)
            if summary_result is not None:
                print("ROBO: " + summary_result)
            else:
                if greeting(user_response) is not None:
                    print("ROBO: " + greeting(user_response))
                else:
                    print("ROBO: ", end="")
                    print(respond(user_response))
                    if user_response in sent_tokens:
                        sent_tokens.remove(user_response)
    else:
        flag = False
        print("ROBO: Bye! Take care.")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
ROBO: I am glad! You are talking to me
ROBO: *nods*

Initial number of characters: 966
Number of characters in the summarized text: 369

ROBO: Some chatbots use sophisticated natural language processing systems, 
but many simpler ones scan for keywords within the input, then pull a reply with the most matching keywords, 
or the most similar wording pattern, from a database.

 
A chatbot or chatterbot is a software application aimed at simulating conversation with human users, especially over the Internet. 

